Exploratory analysis for the steam data.  Goal: predict % positive reviews all-time from all other features.

Continuous vars: reviews_last_30_days, reviews_all_time, num_achievements, full_price

Plan (for midterm progress):

 - find optimal bins for continuous vars via decision tree
 - apply other transformations for continuous vars (^2, log, log log, winsorize, indicator)
 - set param grid for regular lin reg, elasticnet
 - run
 - visualize errors?
 
Plan (final):

 - add text word counts
 - find optimal bins for text word counts via decision tree
 - apply other transformations for text word counts
 - run
 - visualize errors?

In [1]:
import dataset
import numpy as np
import pandas as pd
import sklearn.linear_model
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.model_selection

db = dataset.connect('postgres:///steam')

Pull the data out of the database.  Take all non-DLC games with the target not being null.

In [3]:
query = '''
SELECT
  steam_app_id,
  game_name,
  reviews_last_30_days,
  pct_positive_reviews_last_30_days,
  reviews_all_time,
  pct_positive_reviews_all_time,
  release_date,
  title,
  developer,
  publisher,
  num_achievements,
  full_price,
  metacritic_score,
  genres,
  details,
  tags
FROM
  game_crawl_view
WHERE
  is_dlc = FALSE
  AND pct_positive_reviews_all_time IS NOT NULL
  AND short_description IS NOT NULL
  AND short_description != ''
  AND long_description IS NOT NULL
  AND long_description != '';
'''

data = [r for r in db.query(query)]
df = pd.DataFrame(data)
df.index = df['steam_app_id']
df.drop('steam_app_id', axis=1, inplace=True)
display(df.head())
display(df.shape)

,game_name,reviews_last_30_days,pct_positive_reviews_last_30_days,reviews_all_time,pct_positive_reviews_all_time,release_date,title,developer,publisher,num_achievements,full_price,metacritic_score,genres,details,tags
steam_app_id,,,,,,,,,,,,,,,
10,Counter-Strike,745.0,96.0,39394,97.0,2000-11-01,Counter-Strike,Valve,Valve,NaN,9.99,88.0,[Action],"[Multi-player, Valve Anti-Cheat enabled]","[Action, FPS, Multiplayer, Shooter, Classic, T..."
20,Team Fortress Classic,35.0,71.0,1842,81.0,1999-04-01,Team Fortress Classic,Valve,Valve,NaN,4.99,NaN,[Action],"[Multi-player, Valve Anti-Cheat enabled]","[Classic, Shooter, Class-Based, Team-Based, Fi..."
60,Ricochet,18.0,72.0,1198,79.0,2000-11-01,Ricochet,Valve,Valve,NaN,4.99,NaN,[Action],"[Multi-player, Valve Anti-Cheat enabled]","[Action, FPS, Multiplayer, First-Person, Cyber..."
70,Half-Life,196.0,93.0,11792,96.0,1998-11-08,Half-Life,Valve,Valve,NaN,9.99,96.0,[Action],"[Single-player, Multi-player, Valve Anti-Cheat...","[FPS, Classic, Action, Sci-fi, Singleplayer, S..."
130,Half-Life: Blue Shift,37.0,86.0,1916,90.0,2001-06-01,Half-Life: Blue Shift,Gearbox Software,Valve,NaN,4.99,71.0,[Action],[Single-player],"[FPS, Action, Sci-fi, Singleplayer, Shooter, A..."


(11239, 15)

Create dummy vars containing one-hot encodings for the list variables.

In [4]:
def create_dummies_from_list(df, col, db_table):
    all_values = {r['descr'] for r in db[db_table].find()}
    agg_series = df[col].apply(lambda x: [] if x is None else x)
    df.drop(col, axis=1, inplace=True)
    for val in all_values:
        df['{}|{}'.format(col, val)] = agg_series.apply(lambda x: val in x)

create_dummies_from_list(df, 'genres', 'steam_genre')
create_dummies_from_list(df, 'details', 'steam_game_detail')
create_dummies_from_list(df, 'tags', 'steam_tag')

display(df.head())
display(df.shape)

,game_name,reviews_last_30_days,pct_positive_reviews_last_30_days,reviews_all_time,pct_positive_reviews_all_time,release_date,title,developer,publisher,num_achievements,...,tags|Mature,tags|Silent Protagonist,tags|Trading,tags|Space,tags|Destruction,tags|Superhero,tags|Turn-Based Strategy,tags|Spelling,tags|Education,tags|Pinball
steam_app_id,,,,,,,,,,,,,,,,,,,,,
10,Counter-Strike,745.0,96.0,39394,97.0,2000-11-01,Counter-Strike,Valve,Valve,NaN,...,False,False,False,False,False,False,False,False,False,False
20,Team Fortress Classic,35.0,71.0,1842,81.0,1999-04-01,Team Fortress Classic,Valve,Valve,NaN,...,False,False,False,False,False,False,False,False,False,False
60,Ricochet,18.0,72.0,1198,79.0,2000-11-01,Ricochet,Valve,Valve,NaN,...,False,False,False,True,False,False,False,False,False,False
70,Half-Life,196.0,93.0,11792,96.0,1998-11-08,Half-Life,Valve,Valve,NaN,...,False,True,False,False,False,False,False,False,False,False
130,Half-Life: Blue Shift,37.0,86.0,1916,90.0,2001-06-01,Half-Life: Blue Shift,Gearbox Software,Valve,NaN,...,False,True,False,False,False,False,False,False,False,False


(11239, 409)

Create dummy vars for the categorical variables.  Since some of these have waaay too many values, drop all that are less frequent than 5 obs.

In [5]:
series = df['publisher']
counts = series.value_counts()
counts[counts >= 10]

Ubisoft                                                            138
Strategy First                                                     107
THQ Nordic                                                         100
Square Enix                                                         90
KOEI TECMO GAMES CO., LTD.                                          82
Sekai Project                                                       78
KISS ltd                                                            78
SEGA                                                                66
Paradox Interactive                                                 66
Degica                                                              59
Activision                                                          57
Devolver Digital                                                    57
1C Company                                                          52
Daedalic Entertainment                                              52
Focus 

In [6]:
def create_dummies(df, col, min_num=10):
    series = df[col]
    df.drop(col, axis=1, inplace=True)
    
    counts = series.value_counts()
    filtered_counts = counts[counts >= min_num]
    filtered_series = series.apply(lambda x: x if x in filtered_counts.index else np.NaN)
    return df.merge(pd.get_dummies(filtered_series, prefix=col, prefix_sep='|'), left_index=True, right_index=True)

In [7]:
df = create_dummies(df, 'publisher')
df = create_dummies(df, 'developer')

Look at features highly correlated with the outcome to get an idea of what might be useful.

In [8]:
corr_series = df.corr(min_periods=5)['pct_positive_reviews_all_time'].dropna().sort_values()
display(corr_series[:25])
display(corr_series[-25:])

genres|Simulation                                 -0.144219
tags|Simulation                                   -0.131321
publisher|United Independent Entertainment GmbH   -0.119276
genres|Massively Multiplayer                      -0.104290
tags|Massively Multiplayer                        -0.097987
developer|United Independent Entertainment GmbH   -0.095427
details|MMO                                       -0.093583
genres|Strategy                                   -0.072011
publisher|Strategy First                          -0.071219
publisher|Dagestan Technology                     -0.063234
tags|Strategy                                     -0.060362
publisher|Back To Basics Gaming                   -0.059355
publisher|KOEI TECMO GAMES CO., LTD.              -0.058984
publisher|Libredia                                -0.058945
details|Multi-player                              -0.058210
details|In-App Purchases                          -0.058057
developer|KOEI TECMO GAMES CO., LTD.    

tags|Local Co-Op                     0.087230
tags|Retro                           0.099511
tags|Visual Novel                    0.101571
tags|Female Protagonist              0.103282
tags|Platformer                      0.103403
tags|Anime                           0.110000
tags|Pixel Graphics                  0.115806
tags|Funny                           0.119672
tags|Atmospheric                     0.119681
tags|Comedy                          0.120090
details|Steam Leaderboards           0.120584
tags|Difficult                       0.120869
tags|Cute                            0.132120
tags|Classic                         0.134725
tags|Puzzle                          0.138363
tags|Story Rich                      0.143742
details|Full controller support      0.149747
tags|2D                              0.168572
details|Steam Achievements           0.178800
tags|Singleplayer                    0.183490
details|Steam Cloud                  0.187806
tags|Great Soundtrack             

See which cols have missing values to confirm whether we can just drop them:

In [9]:
for col in df.columns:
    if df[col].dtype == np.float64 and np.NaN in df[col].value_counts(dropna=False).index:
        print(col)

reviews_last_30_days
pct_positive_reviews_last_30_days
num_achievements
full_price
metacritic_score


pct_positive in the last 30 days and metacritic score will be dropped; for the others, "NA" probably means 0.

In [10]:
for col in ['reviews_last_30_days', 'num_achievements', 'full_price']:
    df[col].fillna(value=0, inplace=True)

In [11]:
y = df['pct_positive_reviews_all_time']
y = y.as_matrix()
X = df.drop(['pct_positive_reviews_last_30_days',
             'pct_positive_reviews_all_time',
             'metacritic_score',
             'title', 'game_name', 'release_date'], axis=1)
feature_names = X.columns
X = X.as_matrix()

Fit a basic linear regression and see how it does.

In [12]:
pipeline = sklearn.pipeline.Pipeline((
    ('scaler', sklearn.preprocessing.MinMaxScaler()),
    ('clf', sklearn.linear_model.LinearRegression()),
))

param_grid = {
}

In [13]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size=0.2, random_state=1337)

In [14]:
grid_search = sklearn.model_selection.GridSearchCV(
    pipeline, param_grid=param_grid, scoring='neg_mean_squared_error',
    n_jobs=-1, cv=3)

grid_search.fit(X_train, y_train)

/home/jason/.pyenv/versions/miniconda3-latest/envs/csc522_project/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jason/.pyenv/versions/miniconda3-latest/envs/csc522_project/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jason/.pyenv/versions/miniconda3-latest/envs/csc522_project/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jason/.pyenv/versions/miniconda3-latest/envs/csc522_project/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 b

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=(('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('clf', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)))),
       fit_params=None, iid=True, n_jobs=-1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [15]:
grid_search.best_estimator_.score(X_test, y_test)

-2.5595499314246761e+21

In [16]:
coef_series = pd.Series(grid_search.best_estimator_.named_steps.clf.coef_, index=feature_names).sort_values()
display(coef_series[:25])
display(coef_series[-25:])

genres|Utilities                -4.613633e+14
genres|Software Training        -3.555089e+14
genres|Early Access             -3.704906e+13
tags|Foreign                    -2.568269e+13
genres|Photo Editing            -2.419536e+13
developer|CINEMAX, s.r.o.       -2.312134e+13
publisher|Anarchy Enterprises   -1.758768e+13
publisher|Spiderweb Software    -1.704615e+13
publisher|b-alive gmbh          -1.560645e+13
tags|Software Training          -1.345723e+13
tags|Web Publishing             -1.036079e+13
publisher|Hosted Games          -1.028215e+13
publisher|Lonely Troops         -8.040449e+12
tags|Gaming                     -7.226088e+12
developer|Frozenbyte            -7.019483e+12
developer|Jackbox Games, Inc.   -4.968646e+12
publisher|HeR Interactive       -4.419525e+12
publisher|Last Day of Work      -3.700049e+12
publisher|Arcen Games, LLC      -3.548541e+12
publisher|Sigma Team Inc.       -3.012221e+12
publisher|Winter Wolves         -2.983154e+12
developer|Winged Cloud          -2

publisher|Winged Cloud           2.597238e+12
developer|Winter Wolves          2.983154e+12
developer|Sigma Team Inc.        3.012221e+12
developer|Arcen Games, LLC       3.548541e+12
developer|Last Day of Work       3.700049e+12
developer|HeR Interactive        4.419525e+12
publisher|Jackbox Games, Inc.    4.968646e+12
tags|Photo Editing               5.445608e+12
publisher|Frozenbyte             7.019483e+12
developer|Lonely Troops          8.040449e+12
developer|Hosted Games           1.028215e+13
tags|Audio Production            1.036079e+13
developer|b-alive gmbh           1.560645e+13
developer|Spiderweb Software     1.704615e+13
developer|Anarchy Enterprises    1.758768e+13
publisher|CINEMAX, s.r.o.        2.312134e+13
tags|Mystery Dungeon             2.530300e+13
tags|Early Access                3.704906e+13
details|Downloadable Content     7.802058e+13
genres|Video Production          7.997953e+13
genres|Audio Production          8.048266e+13
genres|Design & Illustration     1